In [ ]:
import pvlib
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime 
import pytz

In [ ]:
times = pd.date_range(start='2015-01-01', end='2015-01-02', freq='1H')

coordinates =[
    (37.98, 23.73, 'Athens', 300, 'Europe/Athens')
]

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

energies = pd.DataFrame(index=times, columns=coordinates)
system = {'module': module, 'inverter': inverter, 'surface_azimuth': 180}

tmys = []

In [ ]:
for location in coordinates:
    latitude, longitude, name, altitude, timezone = location
    system['surface_tilt'] = latitude
    weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude, outputformat='json', map_variables=True)[0]
    #weather = weather.reindex(times)  # Reindex weather data to match desired times
    weather.index.name = "utc_time"
    tmys.append(weather)
    solpos = pvlib.solarposition.get_solarposition(
        time=weather.index,
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        temperature=weather["temp_air"],
        pressure=pvlib.atmosphere.alt2pres(altitude),
    )
    dni_extra = pvlib.irradiance.get_extra_radiation(weather.index)
    airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
    pressure = pvlib.atmosphere.alt2pres(altitude)
    am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
    aoi = pvlib.irradiance.aoi(
        system['surface_tilt'],
        system['surface_azimuth'],
        solpos["apparent_zenith"],
        solpos["azimuth"],
    )
    total_irradiance = pvlib.irradiance.get_total_irradiance(
        system['surface_tilt'],
        system['surface_azimuth'],
        solpos['apparent_zenith'],
        solpos['azimuth'],
        weather['dni'],
        weather['ghi'],
        weather['dhi'],
        dni_extra=dni_extra,
        model='haydavies',
    )
    cell_temperature = pvlib.temperature.sapm_cell(
        total_irradiance['poa_global'],
        weather["temp_air"],
        weather["wind_speed"],
        **temperature_model_parameters,
    )
    effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
        total_irradiance['poa_direct'],
        total_irradiance['poa_diffuse'],
        am_abs,
        aoi,
        module,
    )
    dc = pvlib.pvsystem.sapm(effective_irradiance, cell_temperature, module)
    ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)
    #annual_energy = ac.sum()
    #energies[name] = annual_energy
    #energies[name]= # Append energy values to the list for each location
    energies[location]=ac

#energies

In [ ]:
#weather

In [ ]:
ac.to_csv(r'C:\test\ac.csv', header=True)
ac=pd.read_csv(r"C:\test\ac.csv")
ac['utc_time']=pd.to_datetime(ac['utc_time'])
#ac=ac.set_index('utc_time', inplace=True)
ac = ac.set_index('utc_time').sort_index()

start_date = pd.Timestamp('2013-01-01 00:00:00+00:00')
end_date = pd.Timestamp('2013-01-02 23:00:00+00:00')
ac_new=ac[start_date:end_date]
ac_new.reset_index(inplace=True)
ac_new


In [ ]:
ac_new['utc_time']